# KNN - Defunciones Fetales

En este Jupyter Notebook, se busca crear un modelo de Defunciones Fetales para poder predecir el tipo de defunción fetal que se puede presentar en un embarazo, con base en las características de la madre y del embarazo.

----------
### 1. Importar librerías

In [230]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from kneed import KneeLocator
from sklearn.cluster import KMeans
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from mlxtend.frequent_patterns import apriori
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import f_classif
from prettytable import PrettyTable
import itertools
from sklearn.inspection import permutation_importance
from imblearn.over_sampling import RandomOverSampler

-----------
### 2. Cargar datos
Como primer paso, se cargan los datos ya explorados y limpios. Estos datos se encuentran en el archivo "defunciones_clean.csv".

Las variables presentes son:

    'Área_Geográfica',
    'Asistencia_Recibida',
    'Año_Ocurrencia',
    'Año_Registro',
    'Causa_Defunción',
    'Clase_Parto',
    'Departamento_Ocurrencia',
    'Departamento_Registro',
    'Día_Ocurrencia',
    'Edad_Madre',
    'Estado_Civil_Madre',
    'Escolaridad_Madre',
    'Mes_Ocurrencia',
    'Mes_Registro',
    'Municipio_Ocurrencia',
    'Municipio_Registro',
    'Municipio_Residencia_Madre',
    'Ocupación_Madre',
    'Semanas_Gestacion',
    'Sexo',
    'Sitio_Ocurrencia',
    'Tipo_Atencion',
    'Total_Hijos_Nacidos',
    'Total_Hijos_Nacidos_Muertos',
    'Total_Hijos_Vivos',
    'Via_Parto',
    'Nacionalidad_Madre',
    'Grupo_Etnico_Madre',

In [231]:
# Cargar datos
defunciones = pd.read_csv('defunciones_clean.csv').round(0)


X = defunciones.drop(['Causa_Defunción'], axis=1)
y = defunciones['Causa_Defunción']
oversample = RandomOverSampler(sampling_strategy='minority')
X_over, y_over = oversample.fit_resample(X, y)

defunciones = pd.concat([X_over, y_over], axis=1)


In [232]:
# Eliminar columnas no relevantes (segun analisis exploratorio anteriormente realizado)
defunciones.drop('Municipio_Residencia_Madre', axis=1, inplace=True)

defunciones.drop('Municipio_Registro', axis=1, inplace=True)

defunciones.drop('Municipio_Ocurrencia', axis=1, inplace=True)          

defunciones.drop('Mes_Registro', axis=1, inplace=True)
defunciones.drop('Día_Ocurrencia', axis=1, inplace=True)
defunciones.drop('Mes_Ocurrencia', axis=1, inplace=True)

# Aproximar todos los valores de edad a un entero


defunciones.head()

,Área_Geográfica,Asistencia_Recibida,Año_Ocurrencia,Año_Registro,Clase_Parto,Departamento_Ocurrencia,Departamento_Registro,Departamento_Residencia_Madre,Edad_Madre,Estado_Civil_Madre,...,Sexo,Sitio_Ocurrencia,Tipo_Atencion,Total_Hijos_Nacidos,Total_Hijos_Nacidos_Muertos,Total_Hijos_Vivos,Via_Parto,Nacionalidad_Madre,Grupo_Etnico_Madre,Causa_Defunción
0,99,6,2009,2009,1,7,7,7,18.0,2,...,1,2,1,1.0,2.0,1.0,99,1,4,0
1,99,1,2009,2009,1,13,13,13,36.0,2,...,1,9,1,1.0,3.0,2.0,99,1,4,0
2,99,2,2009,2009,1,2,2,2,22.0,2,...,1,2,1,1.0,3.0,2.0,99,1,4,0
3,99,2,2009,2009,1,5,5,5,26.0,2,...,1,2,1,1.0,0.0,0.0,99,1,4,0
4,99,2,2009,2009,1,7,7,7,43.0,2,...,2,2,1,1.0,10.0,6.0,99,1,4,0


----------
### 3. Selección de variables a utilizar en modelo

En este caso, se busca determinar (a través de SelectKBest y Chi2) las mejores variables para utilizar en el modelo a realizar. Especificamente, se optó por utilizar las 20 variables que tienen una mayor correlación con la variable objetivo (Causa_Defunción).

El código mostrado a continuación selecciona los mejores features del conjunto de datos utilizando la prueba chi-cuadrado (chi-squared test). En concreto, se utiliza la función "SelectKBest" para seleccionar el número "k" de features que se especifican (en este caso, solo el mejor feature) que tienen una mayor relación con la variable objetivo. 

> En otras palabras, se analiza la correlación entre las variables y la variable objetivo, y se seleccionan las variables que tienen una mayor correlación.

In [233]:
# Split the dataset into features (X) and target variable (y)
X = defunciones.drop('Causa_Defunción', axis=1)
y = defunciones['Causa_Defunción']

# Apply feature selection using the chi-squared test
selector = SelectKBest(chi2, k=1)
X_new = selector.fit_transform(X, y)

# Order X_new by feature importance
order = np.argsort(selector.scores_)[::-1]

# Create a list of sorted feature names
feature_names = []
for i in order:
    feature_names.append(X.columns[i])

    if len(feature_names) == 20:
        break

print(feature_names)

X_new = X[feature_names]

['Área_Geográfica', 'Sitio_Ocurrencia', 'Via_Parto', 'Escolaridad_Madre', 'Nacionalidad_Madre', 'Estado_Civil_Madre', 'Tipo_Atencion', 'Clase_Parto', 'Departamento_Residencia_Madre', 'Grupo_Etnico_Madre', 'Sexo', 'Ocupación_Madre', 'Departamento_Registro', 'Departamento_Ocurrencia', 'Asistencia_Recibida', 'Edad_Madre', 'Semanas_Gestacion', 'Total_Hijos_Nacidos_Muertos', 'Total_Hijos_Nacidos', 'Año_Registro']


----------
### 4. Creación del modelo KNN

A continuación se genera el modelo KNN con las variables seleccionadas que tienen mayor correlación con la variable objetivo. Asimismo, se presenta el accuracy del modelo; el cual es 0.89. Esto indica que el modelo es capaz de predecir 89 de 100 veces correctamente el tipo de muerte fetal.

In [234]:
# Split the dataset into training and testing sets
X_new = pd.DataFrame(X_new)
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.3, random_state=42)

# Create the KNN model
knn = KNeighborsClassifier(n_neighbors=5)

# Fit the model to the training data
knn.fit(X_train, y_train)

# Predict on the test set
y_pred = knn.predict(X_test)

# Accuracy del Modelo
print(knn.score(X_test, y_test))

0.9224429131198487


----------
### 5. Proceso Investigativo

En base al modelo generado, se pueden ir alterando ciertos datos para ver cómo se comporta el modelo. Por ejemplo, se puede cambiar la edad de la madre, la escolaridad, el estado civil, etc. y ver cómo se comporta el modelo. Esto con el fin de poder determinar qué características son las que más afectan el resultado de la predicción.


^^^

Como primer paso, se inició determinando qué features tenían más importancia e impacto en el modelo KNN generado. Según este procedimiento, se determinó que el tipo de educacion y via de parto eran los features más importantes. Por lo tanto, se procedió a analizar estos features, al igual que otros, con mayor detalle.

In [235]:
# Obtener la importancia de features
importances, pvalues = f_classif(X, y)

# Imprimir la importancia de cada feature por orden
list_to_order = []
for feature, importance in zip(feature_names, importances):
    list_to_order.append((feature, importance))

list_to_order.sort(key=lambda x: x[1], reverse=True)


x, j = PrettyTable(), []
x.field_names = ["Feature", "Importance"]
for i in list_to_order:
    x.add_row(i)
    j.append(i[0])
    if len(j) == 5:
        break

print(x)

+---------------------+--------------------+
|       Feature       |     Importance     |
+---------------------+--------------------+
|  Escolaridad_Madre  | 1734.8480643425603 |
|      Via_Parto      | 1704.5655494133814 |
|   Área_Geográfica   | 867.3337036139071  |
| Asistencia_Recibida | 765.8761945244026  |
|     Año_Registro    | 716.9410407889744  |
+---------------------+--------------------+


#### 5.1 Escolaridad Madre

In [254]:
# Cual es el nivel de educacion de las madres con Causa_Defunción = 0 = Parto
defunciones_0 = defunciones[defunciones['Causa_Defunción'] == 0]
defunciones_0['Escolaridad_Madre'].value_counts()

1     10999
2     10369
99     6113
3      3991
4      3303
5       493
6         3
Name: Escolaridad_Madre, dtype: int64

In [255]:
# Cual es el nivel de educacion de las madres con Causa_Defunción = 1 = Embarazo
defunciones_1 = defunciones[defunciones['Causa_Defunción'] == 1]
defunciones_1['Escolaridad_Madre'].value_counts()

2     13337
1      9779
3      5064
4      3116
99     2979
5       996
Name: Escolaridad_Madre, dtype: int64

##### Parto -> Embarazo

In [262]:
# Generar nuevo dataset donde Causa_Defunción = 0 = Parto

defunciones_0 = defunciones[defunciones['Causa_Defunción'] == 0]
defunciones_0 = defunciones_0[feature_names+['Causa_Defunción']]

defunciones_0['Escolaridad_Madre'] = 6  #Ninguna Postgrado

# Separar datos prueba y entrenamiento
X_train, X_test, y_train, y_test = train_test_split(defunciones_0.drop('Causa_Defunción', axis=1), defunciones_1['Causa_Defunción'], test_size=0.3, random_state=42)

# Pasar prueba a modelo KNN existente
y_pred = knn.predict(X_test)

# Porcentaje de cambio en la predicción
print("ESCOLARIDAD: Postgrado - Parto")
print("% Cambio", (y_pred != y_test).sum() / len(y_test)*100, "%")

ESCOLARIDAD: Postgrado - Parto
% Cambio 94.88754488754488 %
